# 顧客層に応じた販売戦略（Clustering, 次元圧縮）
教師なし学習
- Clustering
- 次元圧縮

の２つ

## 処理 Pattern と想定される業務利用 Sean
### Clustering
- 顧客を数学的手法に基づき Group 分けする。
- グループ分けの根拠は分析者が後から判断しないといけないが、うまく説明がつけば各Group の特性に基づいた営業戦略を考えらえる

### 次元圧縮
- 何十次元や、何百次元のように項目数が多い入力Data を少ない次元に圧縮する数学的手法。
- Data の行列の行（Case)の類似度で Group化するのが Clustering だとすると、列（項目）の類似度で集約するのが次元圧縮の Image.
- 圧縮先した結果を散布図を表示することで特に重要な顧客を見つけられる可能性がある

Clustering と次元圧縮はよく組み合わせて利用される。